In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import os
from pathlib import Path
import datetime
import random

import dspy

In [3]:
def setup_mlflow():
    import mlflow
    import mlflow.dspy

    mlflow.set_tracking_uri(os.getenv("MLFLOW_TRACKING_URI"))
    experiment = mlflow.set_experiment("dspy-gepa-musique")
    mlflow.dspy.autolog(
        log_compiles=True,
        log_evals=True,
        log_traces_from_compile=True,
    )
    print(f"✅ MLflow tracking enabled at {os.getenv('MLFLOW_TRACKING_URI')}")
    return experiment

mlflow_exp = setup_mlflow()

✅ MLflow tracking enabled at http://localhost:5005


In [4]:
EXP_ID = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
EXP_DIR = Path(f"../outputs/dspy-gepa-musique/{EXP_ID}")
EXP_DIR.mkdir(parents=True, exist_ok=True)
EXP_DIR

PosixPath('../outputs/dspy-gepa-musique/20251003_174554')

In [5]:
# lm = dspy.LM(
#     "openai/Qwen/Qwen2.5-7B-Instruct",
#     temperature=0.6,
#     max_tokens=8192,
#     api_key="local",
#     api_base="http://0.0.0.0:8000/v1",
#     cache=False,
# )
lm = dspy.LM(
    "openai/Qwen/Qwen3-8B",
    temperature=0.6,
    max_tokens=16000,
    api_key="local",
    api_base="http://0.0.0.0:8001/v1",
    cache=False,
)
mlflow_exp._tags["student_lm"] = {
    "model": lm.model,
}
dspy.configure(lm=lm)

reflection_lm = dspy.LM(
    "gemini/gemini-2.5-pro",
    api_key=os.getenv("GEMINI_API_KEY"),
    max_tokens=16384,
    thinking={"type": "enabled"},
    cache=False,
)
mlflow_exp._tags["reflection_lm"] = {
    "model": reflection_lm.model,
}
# reflection_lm = dspy.LM(
#     "openai/Qwen/Qwen3-32B",
#     temperature=0.6,
#     max_tokens=16384,
#     api_key="local",
#     api_base="http://0.0.0.0:8001/v1",
#     cache=False,
# )

In [6]:
lm(messages=[{"role": "user", "content": "Hello"}])

['Hello! How can I assist you today? 😊']

Trace(trace_id=tr-012cd3d8befd4f40a9272e23ae75d2ad)

In [ ]:
reflection_lm(messages=[{"role": "user", "content": "What is largest prime number below 10?"}])

In [7]:
from rlvr.dspy.mhqa.data import prepare_musique_dataset

ds = prepare_musique_dataset(datasets_str="bdsaglam/musique-mini,answerable,train")
random.Random(89).shuffle(ds)
train_size = int(len(ds)*0.80)
train_ds, val_ds = ds[:train_size], ds[train_size:]

test_ds = prepare_musique_dataset(datasets_str="bdsaglam/musique-mini,answerable,validation[:50]")

Map: 100%|##########| 300/300 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Map: 100%|##########| 50/50 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [8]:
from rlvr.dspy.mhqa.baleen import MultiHopQA

program = MultiHopQA(prompt_technique="cot")
program

generate_query.predict = Predict(StringSignature(question, collected_info -> reasoning, search_query, top_n
    instructions='Given a multi-hop question and information collected so far, generate a search query\nto find the next piece of information needed to answer the question.\nFocus on entities, dates, or facts that need to be resolved step by step.'
    question = Field(annotation=str required=True json_schema_extra={'desc': 'The multi-hop question to answer', '__dspy_field_type': 'input', 'prefix': 'Question:'})
    collected_info = Field(annotation=str required=True json_schema_extra={'desc': 'Information collected from previous retrieval steps.', '__dspy_field_type': 'input', 'prefix': 'Collected Info:'})
    reasoning = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${reasoning}', '__dspy_field_type': 'output'})
    search_query = Field(annotation=str required=True json_schema_extra={'desc': 'Search q

In [9]:
example = train_ds[0]
example

Example({'question': "What county contains the work location of the president making father's day a national holiday?", 'answer': 'Washington County', 'answers': ['Washington County', 'washington county'], 'docs': [{'body': 'Thanksgiving, or Thanksgiving Day, is a public holiday celebrated on the fourth Thursday of November in the United States. It originated as a harvest festival. Thanksgiving has been celebrated nationally on and off since 1789, after Congress requested a proclamation by George Washington. It has been celebrated as a federal holiday every year since 1863, when, during the American Civil War, President Abraham Lincoln proclaimed a national day of "Thanksgiving and Praise to our beneficent Father who dwelleth in the Heavens,"to be celebrated on the last Thursday in November. Together with Christmas and the New Year, Thanksgiving is a part of the broader fall / winter holiday season in the U.S.', 'id': '0', 'is_supporting': False, 'text': '# Thanksgiving (United States)

In [10]:
pred = program(example.question, example.docs)
pred

Prediction(
    answer='Washington County, Kansas.',
    collected_info=[KeyInformation(info="President Richard Nixon signed the bill making Father's Day a permanent national holiday in 1972.", source_doc_id='14'), KeyInformation(info="President Richard Nixon signed the bill making Father's Day a permanent national holiday in 1972.", source_doc_id='8'), KeyInformation(info='Washington is the county seat of Washington County, Kansas.', source_doc_id='7')],
    retrieved_doc_ids=['14', '8', '7', '3', '5'],
    citations=['7'],
    n_turns=5
)

Trace(trace_id=tr-32fead41ea23ffbbf6a63af88e0e09f5)

In [11]:
from rlvr.dspy.mhqa.metrics import metric

In [12]:
metric(example, pred)

0.7389189189189189

In [13]:
# Evaluate original program
print("📊 Evaluating ORIGINAL program...")
original_evaluate = dspy.Evaluate(
    devset=test_ds,
    metric=metric,
    num_threads=16,
    display_table=False,
    display_progress=True
)
original_eval_result = original_evaluate(program)

📊 Evaluating ORIGINAL program...
Average Metric: 32.62 / 50 (65.2%): 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [01:37<00:00,  1.95s/it]

2025/10/03 17:48:41 INFO dspy.evaluate.evaluate: Average Metric: 32.61968893468664 / 50 (65.2%)



🏃 View run eval at: http://localhost:5005/#/experiments/1/runs/a899584fa46e497c90e69a89e7afbed5
🧪 View experiment at: http://localhost:5005/#/experiments/1


[Trace(trace_id=tr-ff07ff644edd5c4fcbf1d83a5a1904b3), Trace(trace_id=tr-e9f88581ab76d46038816e22623cf053), Trace(trace_id=tr-57a65ce2e9c604465e43535860594585), Trace(trace_id=tr-c9518df5f3ad3e850de268c1303b6d7a), Trace(trace_id=tr-6b4122e5f7ecd0cb054fe84ee9fa7869), Trace(trace_id=tr-3f334bb0389211aacc5e090d22532855), Trace(trace_id=tr-fe1d5c76461fd946e4ba6924ae3e579d), Trace(trace_id=tr-d1ff50fc6244ca5dfbafbc4cb678efb6), Trace(trace_id=tr-7d95d4597d00fdbbbaf38a6b37c6d2fd), Trace(trace_id=tr-ebbf5666bff98b3f4c6144eb943a4641)]

In [14]:
# raise ValueError()

## GEPA Optimization

GEPA is a reflective prompt optimizer that uses textual feedback to improve performance. We'll create feedback functions for each evaluation aspect and optimize our multi-hop QA program.


In [15]:
from rlvr.dspy.mhqa.feedback_metrics import metric_with_feedback

In [16]:
# Test the feedback metric on our example
feedback_result = metric_with_feedback(example, pred)
print(f"Score: {feedback_result.score:.3f}")
print(f"Feedback: {feedback_result.feedback}")


Score: 0.739
Feedback: Overall performance breakdown:
- Answer F1 Score: Good partial match (F1: 0.80). Your answer contains relevant information but could be more complete or precise. Consider including more specific details from the retrieved documents.
- Retrieval Recall: Perfect retrieval! You found all 3 supporting documents: ['14', '5', '7']
- Retrieval Precision: Moderate precision (precision: 0.60). Only 3 out of 5 retrieved documents are relevant. Many irrelevant docs retrieved: ['3', '8']. Your search queries are too broad - focus on more specific terms and entities.
- Citations F1 Score: Good citations (F1: 0.50). Correct: ['7']. Missing: ['14', '5']. Be more precise about which documents actually support your answer.
- Hop Efficiency: Moderate efficiency (penalty: 0.64). You took 5 turns vs reference 3 hops. Your retrieval queries may be too specific or missing key entities. Try broader, more comprehensive initial searches.


In [17]:
from dspy import GEPA

# Set up GEPA optimizer with reflection LM for optimization
optimizer = GEPA(
    metric=metric_with_feedback,
    auto="light",  # Use light budget for faster experimentation. Use "heavy" for best performance
    num_threads=16,
    track_stats=True,
    use_merge=False,
    reflection_lm=reflection_lm  
)

print("✅ GEPA optimizer configured")

✅ GEPA optimizer configured


In [ ]:
# Run GEPA optimization
print("🚀 Starting GEPA optimization...")

optimized_program = optimizer.compile(
    program,
    trainset=train_ds,
    valset=val_ds,
)

print("✅ GEPA optimization completed!")

2025/10/03 17:48:42 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'cc25b43752d045cc89b5dfdce780ed02', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current dspy workflow
2025/10/03 17:48:42 INFO dspy.teleprompt.gepa.gepa: Running GEPA for approx 2065 metric calls of the program. This amounts to 6.88 full evals on the train+val set.
2025/10/03 17:48:42 INFO dspy.teleprompt.gepa.gepa: Using 60 examples for tracking Pareto scores. You can consider using a smaller sample of the valset to allow GEPA to explore more diverse solutions within the same budget.


🚀 Starting GEPA optimization...


GEPA Optimization:   0%|                                                                                                                              | 0/2065 [00:00<?, ?rollouts/s]2025/10/03 17:51:24 INFO dspy.evaluate.evaluate: Average Metric: 36.27238962477294 / 60 (60.5%)


🏃 View run eval_0 at: http://localhost:5005/#/experiments/1/runs/c98283f27394440dba15b7184f9cdf9c
🧪 View experiment at: http://localhost:5005/#/experiments/1


2025/10/03 17:51:24 INFO dspy.teleprompt.gepa.gepa: Iteration 0: Base program full valset score: 0.6045398270795489
GEPA Optimization:   3%|███▎                                                                                                               | 60/2065 [02:42<1:30:35,  2.71s/rollouts]2025/10/03 17:51:24 INFO dspy.teleprompt.gepa.gepa: Iteration 1: Selected program 0 score: 0.6045398270795489


...

2025/10/03 19:06:46 INFO dspy.teleprompt.gepa.gepa: Iteration 19: New subsample score is not better, skipping
GEPA Optimization:  43%|████████████████████████████████████████████████▍                                                               | 894/2065 [1:18:04<2:07:42,  6.54s/rollouts]2025/10/03 19:06:46 INFO dspy.teleprompt.gepa.gepa: Iteration 20: Selected program 0 score: 0.6045398270795489


Average Metric: 1.66 / 3 (55.2%): 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:59<00:00, 19.87s/it]

2025/10/03 19:07:46 INFO dspy.evaluate.evaluate: Average Metric: 1.6564504504504503 / 3 (55.2%)



🏃 View run eval_51 at: http://localhost:5005/#/experiments/1/runs/2ef85da2b8aa4756b4c17564e516f5db
🧪 View experiment at: http://localhost:5005/#/experiments/1


2025/10/03 19:08:27 INFO dspy.teleprompt.gepa.gepa: Iteration 20: Proposed new text for extract_info.predict: Your task is to act as a reasoning and information extraction module. Given a user's question and a set of retrieved text documents, you must carefully analyze the documents and extract only the key pieces of information that directly help in answering the question.

### Core Instructions:

1.  **Deconstruct the Question:** First, break down the user's question into its core components. Identify the specific entities (people, places, organizations), relationships, and the type of information being asked for (e.g., a date, a reason, a method). Be wary of convoluted phrasing or potential red herrings that are not addressed in the provided documents.

2.  **Targeted and Atomic Extraction:** Scan the documents for information that directly addresses the components of the question. Your goal is not to summarize the documents, but to pull out specific, relevant facts.
    *   **Atomi

In [ ]:
optimized_program.save(str(EXP_DIR / "optimized-program"), save_program=True)

### Examine Optimized Prompts

Let's look at how GEPA improved the prompts for each predictor:


In [ ]:
for name, pred in optimized_program.named_predictors():
    print("=" * 60)
    print(f"Predictor: {name}")
    print("=" * 60)
    print("Optimized Instructions:")
    print(pred.signature.instructions)
    print("*" * 60)


### Evaluate Optimized Program

Compare the performance before and after GEPA optimization:


In [ ]:
print("\\n📊 Evaluating OPTIMIZED program...")
# Evaluate optimized program  
optimized_evaluate = dspy.Evaluate(
    devset=test_ds,
    metric=metric,
    num_threads=8,
    display_table=False,
    display_progress=True
)
optimized_eval_result = optimized_evaluate(optimized_program)

In [42]:
print("=" * 50)
print("🏆 PERFORMANCE COMPARISON")
print("=" * 50)
print(f"Original Program Score:  {original_eval_result.score:.3f}")
print(f"Optimized Program Score: {optimized_eval_result.score:.3f}")
print(f"Improvement:             {optimized_eval_result.score - original_eval_result.score:+.3f}")
print(f"Relative Improvement:    {((optimized_eval_result.score / original_eval_result.score) - 1) * 100:+.1f}%")

🏆 PERFORMANCE COMPARISON
Original Program Score:  65.240
Optimized Program Score: 73.440
Improvement:             +8.200
Relative Improvement:    +12.6%


### GEPA Optimization Analysis

Analyze the detailed optimization results:


In [26]:
# Analyze GEPA optimization trajectory
if hasattr(optimized_program, 'detailed_results'):
    results = optimized_program.detailed_results
    
    print("🔍 GEPA Optimization Details:")
    print(f"- Total candidates explored: {len(results.candidates)}")
    print(f"- Best candidate index: {results.best_idx}")
    print(f"- Best validation score: {results.val_aggregate_scores[results.best_idx]:.3f}")
    print(f"- Discovery evaluations used: {sum(results.discovery_eval_counts)}")
    
    # Show score progression
    print("\\n📈 Score progression:")
    for i, score in enumerate(results.val_aggregate_scores[:10]):  # Show first 10
        print(f"Candidate {i}: {score:.3f}")
    
    if len(results.val_aggregate_scores) > 10:
        print(f"... and {len(results.val_aggregate_scores) - 10} more candidates")
else:
    print("Detailed results not available (set track_stats=True in GEPA constructor)")


🔍 GEPA Optimization Details:
- Total candidates explored: 30
- Best candidate index: 19
- Best validation score: 0.686
- Discovery evaluations used: 30432
\n📈 Score progression:
Candidate 0: 0.605
Candidate 1: 0.575
Candidate 2: 0.587
Candidate 3: 0.590
Candidate 4: 0.469
Candidate 5: 0.582
Candidate 6: 0.476
Candidate 7: 0.674
Candidate 8: 0.585
Candidate 9: 0.636
... and 20 more candidates


In [39]:
# def dag_to_dot(parent_program_for_candidate, dominator_program_ids, best_program_idx, full_eval_scores):
#     dot_lines = [
#         "digraph G {",
#         "    node [style=filled, shape=circle, fontsize=50];"
#     ]
#     n = len(parent_program_for_candidate)
#     # Set up nodes with colors and scores in labels
#     for idx in range(n):
#         score = full_eval_scores[idx]
#         label = f"{idx}\\n({score:.2f})"
#         if idx == best_program_idx:
#             dot_lines.append(f'    {idx} [label="{label}", fillcolor=cyan, fontcolor=black];')
#         elif idx in dominator_program_ids:
#             dot_lines.append(f'    {idx} [label="{label}", fillcolor=orange, fontcolor=black];')
#         else:
#             dot_lines.append(f'    {idx} [label="{label}"];')
    
#     # Set up edges
#     for child, parents in enumerate(parent_program_for_candidate):
#         for parent in parents:
#             if parent is not None:
#                 dot_lines.append(f'    {parent} -> {child};')
    
#     dot_lines.append("}")
#     return "\n".join(dot_lines)

# from gepa.gepa_utils import find_dominator_programs
# pareto_front_programs = find_dominator_programs(optimized_program.detailed_results.per_val_instance_best_candidates, optimized_program.detailed_results.val_aggregate_scores)

# print(dag_to_dot(
#     optimized_program.detailed_results.parents,
#     pareto_front_programs,
#     optimized_program.detailed_results.best_idx,
#     optimized_program.detailed_results.val_aggregate_scores
# ))

In [32]:
# Test optimized program on the same example
example = test_ds[2]

print("🧪 Testing optimized program on example:")
print(f"Question: {example.question}")
print(f"Expected Answer: {example.answer}")
print(f"Supporting Docs: {example.supporting_ids}")
print()

pred = program(example.question, example.docs)
original_metric_result = metric_with_feedback(example, pred)

print("📋 ORIGINAL vs OPTIMIZED Results:")
print("-" * 50)
print("ORIGINAL:")
print(f"  Answer: {pred.answer}")
print(f"  Retrieved docs: {pred.retrieved_doc_ids}")
print(f"  Citations: {pred.citations}")
print(f"Original score: {original_metric_result.score:.3f}")
# print(f"Original feedback: {original_metric_result.feedback}")

optimized_pred = optimized_program(example.question, example.docs)
optimized_metric_result = metric_with_feedback(example, optimized_pred)
print("OPTIMIZED:")
print(f"  Answer: {optimized_pred.answer}")
print(f"  Retrieved docs: {optimized_pred.retrieved_doc_ids}")
print(f"  Citations: {optimized_pred.citations}")
print(f"Optimized score: {optimized_metric_result.score:.3f}")
# print(f"Optimized feedback: {optimized_metric_result.feedback}")

🧪 Testing optimized program on example:
Question: Who is played by the director of The Good Shepherd in The Godfather?
Expected Answer: Vito Corleone
Supporting Docs: ['10', '11']

📋 ORIGINAL vs OPTIMIZED Results:
--------------------------------------------------
ORIGINAL:
  Answer: The question cannot be answered based on the provided information, as it conflates the director of *The Good Shepherd* (Robert De Niro) with the director of *The Godfather* (Francis Ford Coppola).
  Retrieved docs: ['10', '9', '11']
  Citations: ['10', '9', '11']
Original score: 0.568
OPTIMIZED:
  Answer: Vito Corleone
  Retrieved docs: ['10', '9', '11']
  Citations: ['9', '10', '11']
Optimized score: 0.838


[Trace(trace_id=tr-103a2c8e6324e0a571e64a2dccdb933b), Trace(trace_id=tr-eb4c45f093ee951d34a2b6c3871db411)]

Can we measure instruction quality by using them with a larger model to see if it gets questions right?

In [33]:
optimized_eval_result

EvaluationResult(score=73.44, results=<list of 50 results>)

In [36]:
print("\\n📊 Evaluating OPTIMIZED program with teach LM...")
with dspy.context(lm=reflection_lm):
    # Evaluate optimized program  
    teacher_optimized_evaluate = dspy.Evaluate(
        devset=test_ds,
        metric=metric,
        num_threads=8,
        display_table=False,
        display_progress=True
    )
    
    teacher_optimized_eval_result = optimized_evaluate(optimized_program)

\n📊 Evaluating OPTIMIZED program with teach LM...
Average Metric: 36.80 / 50 (73.6%): 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [14:05<00:00, 16.91s/it]

2025/10/04 15:07:22 INFO dspy.evaluate.evaluate: Average Metric: 36.796766974766975 / 50 (73.6%)



🏃 View run eval at: http://localhost:5005/#/experiments/1/runs/5a516d8318a54670be615d3aedcfe306
🧪 View experiment at: http://localhost:5005/#/experiments/1


[Trace(trace_id=tr-0105d067718a764f14c8aed08ce3ba29), Trace(trace_id=tr-69113a1f125a4f81a0b552f415520b77), Trace(trace_id=tr-9cf4f3cfe27136ab8d88d5db113aaa69), Trace(trace_id=tr-53725afeb9f5746764aa90cb41cbd3b9), Trace(trace_id=tr-27bc925f84b0c12cbd5803f0bfc2e1f8), Trace(trace_id=tr-0f43ab31c714bf0611a2ad4a5d32c0fe), Trace(trace_id=tr-7e827ed39ee3386468c00673000a2d26), Trace(trace_id=tr-3f4a8b69afcda200c1f50294ee71f63e), Trace(trace_id=tr-431dddcec67ed96b6e5939b5ea882c14), Trace(trace_id=tr-7742bb80d5c455fcf1f00c7872fba856)]